In [1]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Optimizing Joins")
    .master("spark://0b8ee91998c1:7077")
    .config("spark.cores.max", 4)
    .config("spark.executor.cores", 2)
    .config("spark.executor.memory", "512M")
    .getOrCreate()
)

spark

In [2]:
# Disable AQE and Broadcast join

spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

#### Join Big and Small table - SortMerge vs BroadCast Join

In [3]:
# Read EMP CSV data

_schema = "first_name string, last_name string, job_title string, dob string, email string, phone string, salary double, department_id int"

emp = spark.read.format("csv").schema(_schema).option("header", True).load("data/input/join_data/employee_records.csv")

In [4]:
# Read DEPT CSV data

_dept_schema = "department_id int, department_name string, description string, city string, state string, country string"

dept = spark.read.format("csv").schema(_dept_schema).option("header", True).load("data/input/join_data/department_data.csv")

In [5]:
# Join Datasets
from pyspark.sql.functions import broadcast

# Sort Merge Join
df_joined = emp.join(dept, on=emp.department_id==dept.department_id, how="left_outer")

In [6]:
# Join Datasets
from pyspark.sql.functions import broadcast

# Broadcast Hash Join
# If you are joining a smaller and bigger dataset, Broadcast Joins are more optimal
# Best to avoid broadcasting data larger than 1 GB
df_joined = emp.join(broadcast(dept), on=emp.department_id==dept.department_id, how="left_outer")

In [7]:
df_joined.write.format("noop").mode("overwrite").save()

#### Join Big and Big table - SortMerge without Buckets

Bucketing in Spark is an optimization technique that divides data into a fixed number of "buckets" based on the hash value of one or more specified columns. The main goal of bucketing is to improve the efficiency of join and aggregation operations on large datasets by **avoiding data shuffling** and enabling Spark to read only relevant data when performing these operations.

In [17]:
# Read Sales data

sales_schema = "transacted_at string, trx_id string, retailer_id string, description string, amount double, city_id string"

sales = spark.read.format("csv").schema(sales_schema).option("header", True).load("data/input/join_data/new_sales.csv")

In [18]:
# Read City data

city_schema = "city_id string, city string, state string, state_abv string, country string"

city = spark.read.format("csv").schema(city_schema).option("header", True).load("data/input/join_data/cities.csv")

In [19]:
# Join Data

df_sales_joined = sales.join(city, on=sales.city_id==city.city_id, how="left_outer")

In [20]:
df_sales_joined.write.format("noop").mode("overwrite").save()

In [21]:
# Notice that a Shuffle (Exchange) is performed as part of the join
# In a Spark explain plan, "Exchange" refers to a physical operation where data is shuffled or redistributed across different partitions or nodes in the cluster,
df_sales_joined.explain()

== Physical Plan ==
*(4) SortMergeJoin [city_id#87], [city_id#88], LeftOuter
:- *(1) Sort [city_id#87 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(city_id#87, 200), ENSURE_REQUIREMENTS, [plan_id=292]
:     +- FileScan csv [transacted_at#82,trx_id#83,retailer_id#84,description#85,amount#86,city_id#87] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/data/input/join_data/new_sales.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<transacted_at:string,trx_id:string,retailer_id:string,description:string,amount:double,cit...
+- *(3) Sort [city_id#88 ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(city_id#88, 200), ENSURE_REQUIREMENTS, [plan_id=304]
      +- *(2) Filter isnotnull(city_id#88)
         +- FileScan csv [city_id#88,city#89,state#90,state_abv#91,country#92] Batched: false, DataFilters: [isnotnull(city_id#88)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/data/input/

##### Write Sales and City data in Buckets

In [7]:
# Write Sales data in Buckets

sales.write.format("csv").mode("overwrite").bucketBy(4, "city_id").option("header", True).option("path", "data/output/join_data/sales_bucket.csv").saveAsTable("sales_bucket")

In [8]:
# Write City data in Buckets

city.write.format("csv").mode("overwrite").bucketBy(4, "city_id").option("header", True).option("path", "data/output/join_data/city_bucket.csv").saveAsTable("city_bucket")

In [9]:
# Check tables

spark.sql("show tables in default").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|  default| city_bucket|      false|
|  default|sales_bucket|      false|
+---------+------------+-----------+



#### Join Sales and City data - SortMerge with Bucket

In [10]:
# Read Sales table

sales_bucket = spark.read.table("sales_bucket")

In [11]:
# Read City table

city_bucket = spark.read.table("city_bucket")

In [12]:
# Join datasets

df_joined_bucket = sales_bucket.join(city_bucket, on=sales_bucket.city_id==city_bucket.city_id, how="left_outer")

In [13]:
# Write dataset
# Notice that Shuffle is avoided

df_joined_bucket.write.format("noop").mode("overwrite").save()

In [14]:
# Notice that Shuffle is avoided
df_joined_bucket.explain()

== Physical Plan ==
*(3) SortMergeJoin [city_id#63], [city_id#64], LeftOuter
:- *(1) Sort [city_id#63 ASC NULLS FIRST], false, 0
:  +- FileScan csv spark_catalog.default.sales_bucket[transacted_at#58,trx_id#59,retailer_id#60,description#61,amount#62,city_id#63] Batched: false, Bucketed: true, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/spark-warehouse/data/output/join_data/sales_bucket.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<transacted_at:string,trx_id:string,retailer_id:string,description:string,amount:double,cit..., SelectedBucketsCount: 4 out of 4
+- *(2) Sort [city_id#64 ASC NULLS FIRST], false, 0
   +- *(2) Filter isnotnull(city_id#64)
      +- FileScan csv spark_catalog.default.city_bucket[city_id#64,city#65,state#66,state_abv#67,country#68] Batched: false, Bucketed: true, DataFilters: [isnotnull(city_id#64)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/spark-warehouse/data/output/join_data

In [ ]:
# View how tasks are reading Bucket data



#### Notes